In [19]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
from lxml import html
import scrapy
from time import sleep
import urllib3
import json
from selenium import webdriver
import random

In [20]:

"""
Diferentes parsers según estructura de la página
"""
def parser(link):

    encabezados = {
    'user-agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/44.0.2403.157 Safari/537.36'
    }

    resp = requests.get(link, headers=encabezados, verify=False)
    resp = resp.text

    #soup = get_Soup('https://minciencias.gov.co/convocatorias/todas')
    parser = html.fromstring(resp)

    return parser

def links_pag_semilla(parser):
    links = parser.xpath('//div[@class="contenido-pronaces"]/ul/li//@href')
    urls = []
    url = 'https://www.conacyt.gob.mx'
    
    for link in links:
        if link != '#' and len(link) != 0 and link[0:7] != '/Avisos':
            urls.append(url+link)
    
    return urls

def links_pag_segundo(parser):
    
    links = parser.xpath('//div[@class="contenido-pronaces"]/ul/li//@href')
    urls = []
    url = 'https://www.conacyt.gob.mx'
    
    for link in links:
        if link != '#' and len(link) != 0 and link[-4:] != '.pdf' and link[-5:] != '.docx':
            urls.append(url+link)
        else:
            urls=[]

    return urls


### Datos ####


def titulo(parser):
    texts = parser.xpath('//div[@class="contenido-pronaces"]//text()')

    titulo = []
    for text in texts:
        if len(text) > 50:
            titulo.append(text)

    return titulo[0]

def titulo2(parser):
    """
    Esta solo se ejecuta en caso de que no se peuda extraer el texto con el primero
    """
    texts = parser.xpath('//div[@class="contenido-pronaces"]//text()')

    txt = []
    for text in texts:
        text = text.strip()
        
        if len(text) != 0:
            txt.append(text)

    return txt[0]

def datos_proyecto(parser):
    datos = {}

    try:
        
        for i in range(2,7):
            dato_conv = parser.xpath('//table//tr['+str(i)+']/td[2]/text()')
            if i == 2:
                datos['Publicaicón de la Convocatoria'] = dato_conv[0]
            if i == 3:
                datos['Periodo Recepción de Propuestas'] = dato_conv[0]
            if i == 4:
                datos['Periodo Evaluación'] = dato_conv[0]
            if i == 5:
                datos['Publicación de Resultados'] = dato_conv[0]
            if i == 6:
                datos['Periodo de Formalización'] = dato_conv[0]
    except IndexError:
        datos['Publicaicón de la Convocatoria'] = ''
        datos['Periodo Recepción de Propuestas'] = ''
        datos['Periodo Evaluación'] = ''
        datos['Publicación de Resultados'] = ''
        datos['Periodo de Formalización'] = ''



    return datos

def link_pdf(parser):
    pdfs = parser.xpath('//ul[@class="avisos"]//li/a/@href')
    link = 'https://www.conacyt.gob.mx'

    urls = ''
            

    for pdf in pdfs:
        pdf = link+pdf
        urls = urls + pdf + ', '
    
    return urls

def descripcion(parser):    
    descripcion = parser.xpath('//div[@class="contenido-pronaces"]//p/text()')
    desc = ''
    for i in descripcion:
        i = i.split('\r\n')
        text = ''
        
        for x in i:
            text = text + x.strip() + ' '
        
        desc = desc + text.strip() + ' '
        desc = desc.strip()
    if desc.startswith('Se solicita tomar'):
        desc = ''

    return desc





#########################################
#########################################
#########################################


In [21]:
def Mexico():
    parser_semilla = parser('https://www.conacyt.gob.mx/Convocatorias-y-resultados.html')
    links_pag = links_pag_semilla(parser_semilla)

    parser_pag_segundo = parser(links_pag[0])
    links_pag2 = links_pag_segundo(parser_pag_segundo)

    titulos = []
    pdfs = []  
    links = []
    publicaciones = []
    recepciones = []
    evaluaciones = []
    resultados = []
    formalizaciones = []
    descripciones = []

    for link2_segundo in links_pag2:

        parser_proyecto = parser(link2_segundo)
        #links_pdf = link_pdf()
            
        titulot = titulo(parser_proyecto)
        pdf = link_pdf(parser_proyecto)
        datos_proyectot = datos_proyecto(parser_proyecto)
        descripciont = descripcion(parser_proyecto)
        
        titulos.append(titulot)
        pdfs.append(pdf)
        links.append(link2_segundo)
        descripciones.append(descripciont)

        publicaciones.append(datos_proyectot['Publicaicón de la Convocatoria'])
        recepciones.append(datos_proyectot['Periodo Recepción de Propuestas'])
        evaluaciones.append(datos_proyectot['Periodo Evaluación'])
        resultados.append(datos_proyectot['Publicación de Resultados'])
        formalizaciones.append(datos_proyectot['Periodo de Formalización'])

    mexico = pd.DataFrame()
    mexico['Título'] = titulos
    mexico['Link Proyecto'] = links
    mexico['PDFS'] = pdfs
    mexico['Descripción'] = descripciones 
    mexico['Públicaciones de la convocatoria'] = publicaciones
    mexico['Periodo Recepción de Propuestas'] = recepciones
    mexico['Periodo Evaluación'] = evaluaciones
    mexico['Publicación de Resultados'] = resultados
    mexico['Periodo de Formalización'] = formalizaciones

    return mexico



mexico = Mexico()
mexico.to_excel('Mexico.xlsx')